<a href="https://colab.research.google.com/github/201921027/graduate_project_git/blob/main/gp_new_model_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kogpt fine-tuning classification

https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-5.%20kogpt2_kornli_tpu.ipynb

summarize 코드 참조

https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/

kakaoBrain kogpt3

https://github.com/kakaobrain/kogpt 

skt-ai kogpt2

https://github.com/SKT-AI/KoGPT2

runtime: ram 확장, GPU 가속

* **데이터 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

/content/drive/MyDrive/Colab Notebooks/graduation_project


In [ ]:
import csv

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_total+2.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv', 'r') as data:
  test_data= csv.reader(data, delimiter=',')

In [ ]:
import pandas as pd

# pd.reset_option('display.max_colwidth', None)

train_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/train_total+2.csv') # encoding='cp949'
test_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data.csv')

In [ ]:
train_df= train_df[['title', 'text', 'emotion', 'emotion & summarize']]
test_df= test_df[['title', 'text', 'emotion']]
# train_df.head()

In [ ]:
train_df['emotion'].value_counts()

기쁨         834
슬픔         606
두려움        176
분노         168
unknown    150
Name: emotion, dtype: int64

In [ ]:
test_df['emotion'].value_counts()

기쁨         15
슬픔          5
unknown     5
분노          4
두려움         2
Name: emotion, dtype: int64

In [ ]:
train_df['train_text']=    "["+ train_df['title'].map(str) + "] " + train_df['text']  + ' |' + train_df['emotion']
train_df['train_summary'] =  train_df['emotion & summarize']
train_df= train_df[['train_text', 'train_summary']]
test_df['test_text']=  "["+ test_df['title'].map(str) + "] " + test_df['text'] + ' |' + test_df['emotion'] 
test_df= test_df[['test_text']]
# train_df.head()
test_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,test_text
0,[2018년 3월 29일 목요일] 나를 사랑하는 연습 어떻게 하면 나를 사랑할 수...
1,[김동률 오래된콘서트를 기념하며] 김동률이 부르는 노래와 나의 인연은 2015년 ...
2,[성인간호학 중환자실 실습후기] 2017 1월 2일 월요일 수간호사선선생님의 ot...
3,[반포한강에서] 바람이 꽤 쌀쌀해 콧물을 훌쩍이는 날씨에 반포한강에 왔다. 고속버...
4,[햄버거를 먹었다.] 말 그대로 햄버거를 먹었다. 아버지와 둘이서. 원래 아버지는...


In [ ]:
# pd.set_option('display.max_colwidth', None)
# train_df.head(1)

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(gpt)_train_data.csv')
test_df.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(gpt)_test_data.csv')

In [ ]:
train_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(gpt)_train_data.csv')
train_data= train_data[['train_text', 'train_summary']]
# train_data.head()

In [ ]:
# test_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(gpt)_test_data.csv')
# test_data= test_data[['test_text']]

# test section에서 수행

In [ ]:
# 결측값 제거 (단, 중복은 제거하지 않음)
def drop_na(df):
  df= df.dropna()
  df= df.reset_index(drop=True)
  return df

In [ ]:
train_data= drop_na(train_data)

In [ ]:
# gpt2 max length = 1024
# summarize : 200자로 제한
# 본문은 800자로 제한
# summarize는 앞 200자를 자르고, 본문은 앞 400자, 뒤 400자를 잘라서 합침
# 스터디 할 때 어디서인가 들었는데, 
# 모델 용량때문에 데이터를 잘라야할 땐 앞뒤를 잘라서 합치는 게 
# 요약에 가장 성능이 좋다고 들었던 것 같아서...

In [ ]:
max(train_data['train_text'].apply(len))

9306

In [ ]:
i=0
for text in train_data['train_text']:
  if len(text) > 800:
    train_data['train_text'][i] = train_data['train_text'][i][0:400] + train_data['train_text'][i][len(text)-400:len(text)]
  i += 1

In [ ]:
max(train_data['train_text'].apply(len))

800

In [ ]:
max(train_data['train_summary'].apply(len))

289

In [ ]:
i=0
for text in train_data['train_summary']:
  if len(text) > 200:
    train_data['train_summary'][i] = train_data['train_summary'][i][0:200]
  i += 1

In [ ]:
max(train_data['train_summary'].apply(len))

200

In [ ]:
train_data['total']= train_data['train_text'] + ' TL;DR ' + train_data['train_summary']
train_data.head()

,train_text,train_summary,total
0,[걱정] 오늘도 일기를 쓰기 위해 연필을 들었다. 오늘은 2월 5일. 벌써 202...,시간이 많이 흘렀는데 이룬 것이 별로 없는 것 같아 불안하시군요. 미루는 습관 때문...,[걱정] 오늘도 일기를 쓰기 위해 연필을 들었다. 오늘은 2월 5일. 벌써 202...
1,[20141008] 오늘 아침도 여전히 졸렸다. 잠을 적게 자는 것도 아니고 피곤...,엄청 싫어하는 친구가 있어서 짜증이 나는군요. 그 친구가 당신의 친구랑 같이 다녀서...,[20141008] 오늘 아침도 여전히 졸렸다. 잠을 적게 자는 것도 아니고 피곤...
2,[나의 요즘] 사람과 사람이 만났을 때 가장 많이 하는 말은 '요즘 뭐하고 지내?...,누워서만 지내서 몸은 편한데 마음이 불편하군요. 무언가 해야할 것 같은 조바심과 당...,[나의 요즘] 사람과 사람이 만났을 때 가장 많이 하는 말은 '요즘 뭐하고 지내?...
3,[편입일기.. ] 평생을 열심히 살아본 적이 없다고 스스로 생각했다. 성공한 인생...,"미래를 위해 미대 편입학원에 다니기 시작했군요. 하지만 편입 준비를 하는 것도, 학...",[편입일기.. ] 평생을 열심히 살아본 적이 없다고 스스로 생각했다. 성공한 인생...
4,[백수의 하루] 전날 난 일찍자고 일찍 일어나서 일반인의 하루를 보내리라 결심을 ...,취업이 잘 되지 않아 힘든 상황이군요. 직업이 없는 상태라 하루하루 똑같은 날을 보...,[백수의 하루] 전날 난 일찍자고 일찍 일어나서 일반인의 하루를 보내리라 결심을 ...


In [ ]:
# 합쳐진 문장(total) example 전문 출력하여 데이터가 제대로 구성됐는지 확인
# pd.set_option('display.max_colwidth', None)
# train_data['total'].head(1)

In [ ]:
print('total max length: ' , max(train_data['total'].apply(len)))

total max length:  1007


In [ ]:
# !git clone https://github.com/kakaobrain/kogpt.git

In [ ]:
# !git clone https://github.com/SKT-AI/KoGPT2.git

In [ ]:
# ! git clone https://github.com/jiminAn/Capstone_2022.git

* **pre-trained model(kakaoBrain kogpt) 불러오기**

In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/graduation_project/kogpt/requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 831.4 MB 6.5 kB/s 
     |████████████████████████████████| 3.1 MB 37.3 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 880 kB 54.2 MB/s 
     |████████████████████████████████| 596 kB 53.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=79b26bd8b37ee23c289241fd9fc69f1ff5bcdc1fd02a8662ee0cef49b9a77d99
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1

In [ ]:
import torch
import tensorflow
# from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# kogpt3

# pre-trained tokenizer 불러오기
# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
# )

# # pre-trained model 불러오기
# model = AutoModelForCausalLM.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   pad_token_id=tokenizer.eos_token_id,
#   torch_dtype='auto', low_cpu_mem_usage=True
# )#.to(device='cuda', non_blocking=True)
# _ = model.eval()

In [ ]:
# kogpt2
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')


Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/gpt_model.pt').to(device)

In [ ]:
# model test

# text = '[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!'
# input_ids = tokenizer.encode(text, return_tensors='pt')
# gen_ids = model.generate(input_ids,
#                            max_length=128,
#                            repetition_penalty=2.0,
#                            pad_token_id=tokenizer.pad_token_id,
#                            eos_token_id=tokenizer.eos_token_id,
#                            bos_token_id=tokenizer.bos_token_id,
#                            use_cache=True)
# generated = tokenizer.decode(gen_ids[0])
# print(generated)

* 참고한 summarize 코드

https://towardsdatascience.com/question-answering-with-a-fine-tuned-bert-bc4dafd45626

https://medium.com/analytics-vidhya/an-ai-that-does-your-homework-e5fa40c43d17

https://www.nbshare.io/notebook/764386829/Amazon-Review-Summarization-Using-GPT-2-And-PyTorch/

In [ ]:
import numpy as np

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
text_data= train_data['train_text']
summary_data= train_data['train_summary']

In [ ]:
from torch.utils.data import Dataset, DataLoader

max_len= len(max(train_data['total']))
extra_length= len(tokenizer.encode(" TL;DR "))

class textDataset(Dataset):  
    def __init__(self, tokenizer, text, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.text= text
        self.result = []

        for text in self.text:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(text + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [ ]:
dataset= textDataset(tokenizer, train_data['total'], max_len)

In [ ]:
dataloader= DataLoader(dataset, batch_size= 1, drop_last=True)

In [ ]:
# model load
# model= GPT2LMHeadModel.from_pretrained('./checkpoint/gpt_model.pt').to(device)

In [ ]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to(device)
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 100 == 0:
                    print("epoch: %f, loss: %f, %d"%(int(epoch), loss, idx))
                    model.save_pretrained('./checkpoint/gpt_model_1+2.pt') # >>>>>>>>>>>>> epoch마다 모델명 수정할 것 <<<<<<<<<<<< (1)

In [ ]:
import torch.optim as optim

optimizer= optim.AdamW(model.parameters(), lr= 3e-4)
model= model.to(device)

In [ ]:
# train(model=model, optimizer=optimizer, dl=dataloader, epochs=1) # >>>>>>>>>>>>>>>>> epoch 수정 <<<<<<<<<<<<<<<<<< (2)

epoch: 0.000000, loss: 9.516912, 0
epoch: 0.000000, loss: 2.056159, 100
epoch: 0.000000, loss: 1.835126, 200
epoch: 0.000000, loss: 2.224770, 300
epoch: 0.000000, loss: 2.116745, 400
epoch: 0.000000, loss: 2.260252, 500
epoch: 0.000000, loss: 2.048805, 600
epoch: 0.000000, loss: 2.228275, 700
epoch: 0.000000, loss: 2.451510, 800
epoch: 0.000000, loss: 2.289315, 900
epoch: 0.000000, loss: 2.026349, 1000
epoch: 0.000000, loss: 1.811824, 1100
epoch: 0.000000, loss: 1.544204, 1200
epoch: 0.000000, loss: 1.865303, 1300
epoch: 0.000000, loss: 1.942266, 1400
epoch: 0.000000, loss: 1.844208, 1500
epoch: 0.000000, loss: 1.806525, 1600
epoch: 0.000000, loss: 1.560001, 1700
epoch: 0.000000, loss: 1.858768, 1800
epoch: 0.000000, loss: 1.097597, 1900


In [ ]:
# model save
# model.save_pretrained('./checkpoint/gpt_model_1+2.pt') # >>>>>>>>>> epoch마다 모델명 수정할 것 <<<<<<<<< (3)

In [ ]:
# model load
model= GPT2LMHeadModel.from_pretrained('./checkpoint/gpt_model_1+2.pt').to(device) # >>>>>>>>>> epoch마다 모델명 수정할 것 <<<<<<<<< (4)

In [ ]:
# 서버에서 본문 받아오기

text = '[평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다!' # sample
if len(text) + len(" TL;DR ") > 800:
  text = text[0:400] + text[len(text)-400:len(text)]
text= text + " TL;DR "

In [ ]:
# BERT의 output을 파일로 받아와서 str로 저장하기 -> 본문이랑 합쳐서 모델로 넘기기

emotion= '기쁨' # sample
text= emotion + '| ' + text
print(text)

기쁨| [평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다! TL;DR 


In [ ]:
# model overfitting 확인

input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
gen_ids = model.generate(input_ids,
                           max_length=len(input_ids[0])+ 50, # 50 -> 약 120자
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True).to(device)
generated = tokenizer.decode(gen_ids[0])
print(generated)
# print(len(text), len(generated))

기쁨| [평범한 일상] 오늘은 아주 무난한 하루를 보냈다. 팝송도 듣고, 산책도 하고 등등. 매일이 오늘 같았으면 좋겠다! TL;DR  꿈에서 소외감을 느끼는군요. 당신은 항상 우울한 일만 하는 것이니까, 나는 그게 좋은 것 같아요?</s>


In [ ]:
TLDR_index= generated.index('TL;DR')
output= generated[TLDR_index+6:]
print('return comment:' , output)

return comment:  꿈에서 소외감을 느끼는군요. 당신은 항상 우울한 일만 하는 것이니까, 나는 그게 좋은 것 같아요?</s>


* test

In [ ]:
# model load
model= GPT2LMHeadModel.from_pretrained('./checkpoint/gpt_model_1.pt').to(device) # >>>>>>>> epoch마다 모델명 수정할 것 <<<<<<< (5)

In [ ]:
test_data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/new-model(gpt)_test_data.csv')
test_data= test_data[['test_text']]

test_data= drop_na(test_data)

In [ ]:
text_list = test_data['test_text']
summary_list=list()
# print(text_list)

In [ ]:
text_list = test_data['test_text']
summary_list=list()

for i, text in enumerate(text_list):
  if len(text) + len(" TL;DR ") > 800:
    text = text[0:400] + text[len(text)-400:len(text)]
  text= text + " TL;DR "

  input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
  gen_ids = model.generate(input_ids,
                            max_length=len(input_ids[0]) + 120, # 50 -> 약 120자
                            repetition_penalty=2.0,
                            pad_token_id=tokenizer.pad_token_id,
                            eos_token_id=tokenizer.eos_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            use_cache=True).to(device)
  generated = tokenizer.decode(gen_ids[0])
  TLDR_index= generated.index('TL;DR')
  output= generated[TLDR_index+6:]
  summary_list.append(output)
  if i%10 == 0:
    print('test {} 개 진행중'.format(i))

test 0 개 진행중
test 10 개 진행중
test 20 개 진행중
test 30 개 진행중


In [ ]:
summary_list

[' 당신은 분명 나 자신이 될 거예요. 당신의 마음이 답답한 것 같지만, 내가 잘 할 것이라고 생각해요!</s>',
 '괌 여행을 다녀와서 동률을 알려준다는 건 정말 뜻이에요!</s>',
 '괜찮은 상태를 고치느라 힘들었을 것 같아요. 하지만 당신은 분명 좋은 사람인일이었을거라고 생각해요!</s>',
 ' 비가 많이 내려서 우산을 타고 서울역으로 갔군요. 그래도 무사히 도착해서 다행이에요!</s>',
 ' 맛있는 걸로테러 가고, 친구와 함께 놀았군요. 당신은 정말 좋아하는 아이돌 그룹입니다. 하지만 그 순간에 대해 걱정될 일만 했겠어요, 당신의 말을 듣고 기뻤던 건지도 모른다고 생각해서, 좋은 이야기를 해보길 바랄 뿐이에요.</s>',
 ' 새벽밤은 너무 우울한 것 같아요. 당신은 분명 잘 될 것이라고 생각해요?</s>',
 '괜히 마음이 이해가 되는군요. 당신은 분명 좋은 사람이 될 것이라고 생각해요!</s>',
 ' 요즘 당신은 분명 성장할 수 있을 것이라고 생각해요. 당신의 말로는 정말 좋은 경험이 되고 싶어지고, 목표가 되지 않는다면 결국 포기하고 싶은 마음이 이해가 되는군요. 그래도 포기하지 말고 열심히 하길 바랄 뿐입니다.</s>',
 '조나조로 꿈을 꾸고, 공부를 하고 있는군요. 당신은 분명 좋은 사람이 될 거라고 생각해요</s>',
 ' 2019년의 시작과 다짐에 대해 걱정되는군요. 당신은 분명 성장할 수 있을 것이라고 생각해요!</s>',
 ' 당신의 죽음에 대해 걱정하는 건 정말 허무할 일인것 같아요. 당신은 분명 성장할 것이라고 생각해요!</s>',
 ' 첫 임용고시 준비를 하고 나서 좋은 선택을 했군요. 당신은 분명 성장할 거예요!</s>',
 '방 탄북단이 정말 허무하게 몰려왔군요. 하지만 당신의 말대로 좋은 일만 하는 건 어려운 것 같지만, 분명 성장할 수 있을 것이라고 생각해요!</s>',
 ' 해외에 나가니 정말 힘들었을 것 같아요. 당신은 분명 성장할 거예요!</s>',
 '태국에서 여행을 떠나는 걸음으로 비행기를 타고 공항으로

In [ ]:
test_data['summary'] = summary_list
# test_data.head()

In [ ]:
test_data.to_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_newmodel(gpt)1_after.csv') 
# >>>>>>>>>> epoch마다 파일명 수정할 것 <<<<<<<<< (6)

* evaluation

https://github.com/lovit/KoBERTScore

( BERT score )

In [ ]:
# !git clone https://github.com/lovit/KoBERTScore

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore'

/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore


In [ ]:
!pip install -r '/content/drive/MyDrive/Colab Notebooks/graduation_project/KoBERTScore/requirements.txt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.8 MB/s 
     |████████████████████████████████| 61 kB 548 kB/s 
     |████████████████████████████████| 884 kB 29.2 MB/s 
     |████████████████████████████████| 3.0 MB 62.4 MB/s 
     |████████████████████████████████| 1.3 MB 58.7 MB/s 
     |████████████████████████████████| 880 kB 88.2 MB/s 
     |████████████████████████████████| 96 kB 5.5 MB/s 
     |████████████████████████████████| 66 kB 3.4 MB/s 
     |████████████████████████████████| 41 kB 488 kB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7ddda3faa37f82915de34c8826a99373a7c537f436498bce7364f839edcf9c3a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xl

In [ ]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing KoBERTScore.egg-info/PKG-INFO
writing dependency_links to KoBERTScore.egg-info/dependency_links.txt
writing entry points to KoBERTScore.egg-info/entry_points.txt
writing requirements to KoBERTScore.egg-info/requires.txt
writing top-level names to KoBERTScore.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'KoBERTScore.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/__init__.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/tasks.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/about.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/cli.py -> build/bdist.linux-x86_64/egg/KoBERTScore
copying build/lib/KoBERTScore/score.py -> build/

In [ ]:
# %cd '/content/drive/MyDrive/Colab Notebooks/graduation_project'

In [ ]:
import csv
import pandas as pd

# references 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/references.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')

references_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/references.csv') 

references_df= references_df[['title', 'summary']]

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_newmodel(gpt)1_after.csv', 'r') as data:
  train_data= csv.reader(data, delimiter=',')
  # >>>>>>>>>> epoch마다 파일명 수정할 것 <<<<<<<<< (7)

candidates_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graduation_project/data/test_data_newmodel(gpt)1_after.csv') 
  # >>>>>>>>>> epoch마다 파일명 수정할 것 <<<<<<<<< (8)

candidates_df= candidates_df[['test_text', 'summary']]

In [ ]:
references_df.head()

,title,summary
0,2018년 3월 29일 목요일,당신은 예쁜 사람이니까 자책할 필요 없다고 생각해요. 당신을 사랑해줄 사람과 연애를...
1,김동률 오래된콘서트를 기념하며,김동률의 공연을 보며 크게 감명을 받았군요. 인생에서 좋았던 순간들에 한 장면이 추...
2,성인간호학 중환자실 실습후기,중환자실 실습을 했군요. 기관절개 한 곳으로 가래가 넘쳐 나온 걸 연차가 높으신 선...
3,반포한강에서,뱐포 한강에 가서 남준이의 노래를 들었군요. 남준이를 굉장히 좋아하나봐요. 그리고 ...
4,햄버거를 먹었다.,오랜만에 햄버거를 먹었군요! 햄버거를 묘사한 글에서 햄버거에 대한 당신의 애정이 느...


In [ ]:
candidates_df.head()

,test_text,summary
0,[2018년 3월 29일 목요일] 나를 사랑하는 연습 어떻게 하면 나를 사랑할 수...,"당신은 분명 나 자신이 될 거예요. 당신의 마음이 답답한 것 같지만, 내가 잘 할..."
1,[김동률 오래된콘서트를 기념하며] 김동률이 부르는 노래와 나의 인연은 2015년 ...,괌 여행을 다녀와서 동률을 알려준다는 건 정말 뜻이에요!</s>
2,[성인간호학 중환자실 실습후기] 2017 1월 2일 월요일 수간호사선선생님의 ot...,괜찮은 상태를 고치느라 힘들었을 것 같아요. 하지만 당신은 분명 좋은 사람인일이었을...
3,[반포한강에서] 바람이 꽤 쌀쌀해 콧물을 훌쩍이는 날씨에 반포한강에 왔다. 고속버...,비가 많이 내려서 우산을 타고 서울역으로 갔군요. 그래도 무사히 도착해서 다행이에...
4,[햄버거를 먹었다.] 말 그대로 햄버거를 먹었다. 아버지와 둘이서. 원래 아버지는...,"맛있는 걸로테러 가고, 친구와 함께 놀았군요. 당신은 정말 좋아하는 아이돌 그룹입..."


In [ ]:
# bert score
from KoBERTScore import BERTScore

model_name = "beomi/kcbert-base"
bertscore = BERTScore(model_name, best_layer=4)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Load beomi/kcbert-base with 4 layers


In [ ]:
# candidates_df['summary']
# references_df['summary']

In [ ]:
# references : 사람이 만든 요약문
# candidates : 기계가 만든 요약문
score=list()

# list로 들어가야함
references = references_df['summary']
candidates = candidates_df['summary']

score.append(bertscore(references, candidates, batch_size=8))

Calculating BERTScore:   0%|          | 0/4 [00:00<?, ?it/s]
Train IDF: 1it [00:00, 77.26it/s]
Calculating BERTScore: 100%|██████████| 4/4 [00:00<00:00, 22.50it/s]


In [ ]:
import numpy as np

mean_score= np.mean(score[0])

In [ ]:
print(mean_score)
score[0]

0.508772192462798


[0.6613271236419678,
 0.559963047504425,
 0.5623078942298889,
 0.5450291037559509,
 0.4445590674877167,
 0.554026186466217,
 0.35719966888427734,
 0.4983202815055847,
 0.5526773929595947,
 0.47905832529067993,
 0.49657580256462097,
 0.5717474222183228,
 0.4900602102279663,
 0.39512234926223755,
 0.4815613031387329,
 0.5029053688049316,
 0.36121100187301636,
 0.5700283050537109,
 0.41713106632232666,
 0.42842233180999756,
 0.5477308034896851,
 0.5363696217536926,
 0.5427787899971008,
 0.4787497818470001,
 0.5737388730049133,
 0.6413412690162659,
 0.5324189066886902,
 0.5028055310249329,
 0.4910717010498047,
 0.5265694260597229,
 0.4691300094127655]

( BLEU score )

https://www.digitalocean.com/community/tutorials/bleu-score-in-python

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
bl_references = references_df['summary']
bl_candidates = candidates_df['summary']

In [ ]:
bl_references_s= list()
bl_candidates_s= list()

for i in range(len(bl_references)):
  temp= bl_references[i].split()
  bl_references_s.append(temp)

for i in range(len(bl_candidates)):
  temp= bl_candidates[i].split()
  bl_candidates_s.append(temp)

In [ ]:
print(bl_references_s[0])
print(bl_candidates_s[0])

['당신은', '예쁜', '사람이니까', '자책할', '필요', '없다고', '생각해요.', '당신을', '사랑해줄', '사람과', '연애를', '하게', '되기를', '기원할게요.']
['당신은', '정말', '소중한', '사람일', '거예요.', '당신의', '우울한', '일기를', '꾸준히', '써보겠다고', '했겠어요!</s>']


In [ ]:
bleu_score= list()

for i in range(len(bl_candidates_s)):
  bleu_score.append(sentence_bleu(bl_references_s, bl_candidates_s[i]))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
bleu_score

[1.1388294875424009e-231,
 6.5397857717545986e-155,
 1.1388294875424009e-231,
 1.171778691554733e-231,
 1.337299023427136e-231,
 5.8442165442547524e-155,
 5.4227507780138046e-232,
 5.978665957199477e-155,
 5.806795361477156e-155,
 8.212048754125947e-155,
 1.328488253493217e-231,
 6.734343721752972e-232,
 8.313696654408263e-155,
 6.072910347203158e-155,
 5.843526464362002e-155,
 1.2395288183339461e-231,
 6.197842970161413e-155,
 7.690513323747095e-155,
 6.161434453505796e-232,
 5.447474599023752e-155,
 6.164444051541174e-155,
 1.6034157163765524e-231,
 6.9960774230227766e-155,
 4.854408244229234e-155,
 4.393462061369695e-155,
 1.556890273854021e-231,
 5.428795743489114e-155,
 2.250493055448548e-78,
 8.723297561929817e-155,
 5.681181685872076e-155,
 6.577920249432179e-155]